#### KNN Model
n = 7
  
Accuracy = 53%

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,roc_auc_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive,files

In [ ]:
drive.mount("/content/gdrive")
df = pd.read_csv('/content/gdrive/My Drive/580/Combined_Delayed_2Days.csv')
df.head()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,Decrease
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,Increase
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,Decrease
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,Decrease
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,Increase


In [ ]:
df = df[["Date","Tweets_clean_more","Label_delayed_2Days"]]
#df = df.replace({'positive':1,'negative':-1,'neutral':0,'Increase':1,'Decrease':-1,'No Change':0})
df.set_index("Date",inplace= True)
df.head()

,Tweets_clean_more,Label_delayed_2Days
Date,,
2013-01-10,to super clear i dont wish could i mandat anyt...,Decrease
2019-01-10,ye get regulatori pushback may avail region go...,Increase
2020-01-10,dome barrel weld made bar pretti good bar need...,Decrease
2012-01-11,the exec conf room tesla use call denali i dec...,Decrease
2016-01-11,sad hear david bowi die he amaz,Increase


In [ ]:
x = df['Tweets_clean_more']
y = df["Label_delayed_2Days"]
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8, random_state=123)
vec = CountVectorizer()
vec.fit(x_train)
vec.fit(x_test)
x_train_features = vec.transform(x_train)
x_test_features = vec.transform(x_test)

In [ ]:
x_train_features.shape

(1136, 3361)

In [ ]:
neighbors =[i for i in range(1,22,2)]
for n in neighbors:
    clf = KNeighborsClassifier(n_neighbors=n,p=1) # manhattan distance
    clf.fit(x_train_features,y_train)
    y_pred = clf.predict(x_test_features)
    cm = confusion_matrix(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    print("Model report: n = ",n)
    print("Accuracy score: ",acc)
    print("Confusion Matrix:",cm)
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_lb = lb.transform(y_test)
    y_pred_lb = lb.transform(y_pred)
    print("ROC AUC score:", roc_auc_score(y_test_lb,y_pred_lb))
    print("F1 Score:",f1_score(y_test,y_pred,average="micro"))
    print()



Model report: n =  1
Accuracy score:  0.45774647887323944
Confusion Matrix: [[ 26 112   2]
 [ 35 103   1]
 [  1   3   1]]
ROC AUC score: 0.5121442239184976
F1 Score: 0.45774647887323944

Model report: n =  3
Accuracy score:  0.44366197183098594
Confusion Matrix: [[ 17 123   0]
 [ 30 109   0]
 [  0   5   0]]
ROC AUC score: 0.4690848798793477
F1 Score: 0.44366197183098594

Model report: n =  5
Accuracy score:  0.4295774647887324
Confusion Matrix: [[44 96  0]
 [61 78  0]
 [ 1  4  0]]
ROC AUC score: 0.45920434424217604
F1 Score: 0.4295774647887324

Model report: n =  7
Accuracy score:  0.45774647887323944
Confusion Matrix: [[48 92  0]
 [57 82  0]
 [ 0  5  0]]
ROC AUC score: 0.4779977249727312
F1 Score: 0.45774647887323944

Model report: n =  9
Accuracy score:  0.47183098591549294
Confusion Matrix: [[55 85  0]
 [60 79  0]
 [ 0  5  0]]
ROC AUC score: 0.4873076907931782
F1 Score: 0.47183098591549294

Model report: n =  11
Accuracy score:  0.5105633802816901
Confusion Matrix: [[57 83  0]
 [51 

In [ ]:
neighbors =[i for i in range(1,22,2)]
for n in neighbors:
    clf = KNeighborsClassifier(n_neighbors=n,p=2) #euclidean distance
    clf.fit(x_train_features,y_train)
    y_pred = clf.predict(x_test_features)
    cm = confusion_matrix(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    print("Model report: n = ",n)
    print("Accuracy score: ",acc)
    print("Confusion Matrix:",cm)
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_lb = lb.transform(y_test)
    y_pred_lb = lb.transform(y_pred)
    print("ROC AUC score:", roc_auc_score(y_test_lb,y_pred_lb))
    print("F1 Score:",f1_score(y_test,y_pred,average="micro"))
    print()

Model report: n =  1
Accuracy score:  0.5211267605633803
Confusion Matrix: [[ 45  89   6]
 [ 37 102   0]
 [  3   1   1]]
ROC AUC score: 0.555878118629097
F1 Score: 0.5211267605633803

Model report: n =  3
Accuracy score:  0.49295774647887325
Confusion Matrix: [[ 34 106   0]
 [ 34 105   0]
 [  2   2   1]]
ROC AUC score: 0.533904206683914
F1 Score: 0.49295774647887325

Model report: n =  5
Accuracy score:  0.47183098591549294
Confusion Matrix: [[50 90  0]
 [55 84  0]
 [ 4  1  0]]
ROC AUC score: 0.48735849579777885
F1 Score: 0.47183098591549294

Model report: n =  7
Accuracy score:  0.5246478873239436
Confusion Matrix: [[57 83  0]
 [47 92  0]
 [ 4  1  0]]
ROC AUC score: 0.5225893915409544
F1 Score: 0.5246478873239436

Model report: n =  9
Accuracy score:  0.5070422535211268
Confusion Matrix: [[52 88  0]
 [47 92  0]
 [ 3  2  0]]
ROC AUC score: 0.510897866271843
F1 Score: 0.5070422535211268

Model report: n =  11
Accuracy score:  0.5140845070422535
Confusion Matrix: [[54 86  0]
 [47 92  0]
